In [28]:
import os
import sys
import site

cwd = os.getcwd()
print(f"Current directory: {cwd}")
print(f"Current Python version: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print(f"Current Python interpreter: {sys.executable}")
print(f"Current site-packages: {site.getsitepackages()}")

sys.path.append(os.path.join(cwd, "site-packages"))

Current directory: /home/jovyan/docs/source/04-Typing
Current Python version: 3.10.5
Current Python interpreter: /opt/conda/bin/python
Current site-packages: ['/opt/conda/lib/python3.10/site-packages']


In [29]:
# 首先创建一个 Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

# Typing in PySpark

In [30]:
import json
from pathlib import Path
import pyspark.sql.functions as F
import pyspark.sql.types as T

## Mixed Type in One Column

PySpark 的 DataFrame 是 Schema Enforced 的. 也就是说一个 Column 中的值的 Type 必须一致. **如果你是直接创建 DataFrame 而你又 Pass in 了不同的 Type, 那么会直接报错不让你创建**.


In [31]:
pdf = spark.createDataFrame(
    [
        (1, ),
        (2, ),
        (3, ),
        (4, ),
        (5, ),
        ("a", ),
        ("b", ),
    ],
    ("col",)
)
pdf.show()

TypeError: field col: Can not merge type <class 'pyspark.sql.types.LongType'> and <class 'pyspark.sql.types.StringType'>

**如果你是通过 IO 从文件中读取数据, 而存在 Type 不匹配的情况, 那么 Spark 会尝试将所有数据 Convert 成同一个 Type, 通常是字符串. 如果实在不行则报错**. 这个机制也有副作用, 当你期望的数值是 "非字符串" 时, 你往往想要把错误的 "字符串" 所在的行丢弃, 而 Spark 默认的行为则是全部转化成 "字符串".

In [ ]:
p_json = Path(cwd) / "tmp.json"

records = [
    {"col": 1},
    {"col": 2},
    {"col": 3},
    {"col": 4},
    {"col": 5},
    {"col": "a"},
    {"col": "b"},
]

content = "\n".join([
    json.dumps(record)
    for record in records
])

p_json.write_text(content)

pdf = spark.read.json(
    f"{p_json}",
)

pdf.show()

In [ ]:
pdf.printSchema()

In [ ]:
pdf.collect()

为了解决 "错误的数据类型会连累正确的数据类型被强制转化成字符串" 的问题, 你可以在 IO 的时候手动指定 schema, 并允许 ``nullable = True``, 这样凡是数据类型不对的行就会自动被设为 Null. 手动指定 schema 的 API 可以 [参考这里](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.types.StructType.html#pyspark.sql.types.StructType)

In [ ]:
# Define Structure
schema = T.StructType([
    T.StructField("col", T.IntegerType(), True),
])

pdf = spark.read.json(f"{p_json}", schema=schema)

pdf.show()